In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask
import dask.dataframe as dd

from sklearn.model_selection import GridSearchCV, KFold,KFold, cross_val_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler,RobustScaler 

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.linear_model import LogisticRegression,LinearRegression,Lasso

from sklearn.metrics import accuracy_score, f1_score,mean_squared_error,classification_report,confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask
import dask.dataframe as dd

import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import GridSearchCV, KFold,KFold, cross_val_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler,RobustScaler 

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.linear_model import LogisticRegression,LinearRegression,Lasso

from sklearn.metrics import accuracy_score, f1_score,mean_squared_error,classification_report,confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_transformer

from scipy import stats
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline

In [35]:
df =pd.read_pickle("data_after_normalization.pickle")

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1796287 entries, 0 to 1940933
Data columns (total 39 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   park_id                     object        
 1   clc_code                    object        
 2   area_code                   object        
 3   evse_id                     object        
 4   datetime_start              datetime64[ns]
 5   datetime_stop               datetime64[ns]
 6   city                        object        
 7   latitude                    float64       
 8   longitude                   float64       
 9   postal_code                 category      
 10  Charging_Time_in_hours      float64       
 11  year                        category      
 12  month                       category      
 13  day                         category      
 14  day_of_week_name_Start      object        
 15  Precipitation               float64       
 16  Temperature       

# Generalized Least Squares with AR covariance structure

In [192]:
Y= df[["Charging_Time_in_hours"]]

In [193]:
X = df[["pd_New","Precipitation_New","Temperature_New","Parking_New","university_New","Biergarten_New","Kino_New","Restaurant_New","Community_Centre_New","Hospital_New","Bahnhof_New",]]

In [194]:
X = sm.add_constant(X)

/home/col-gku/.local/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [201]:
model = sm.GLS(Y, X, rho=2)

In [211]:
res_wls = model.fit()

In [212]:
print(res_wls.summary())

                              GLS Regression Results                              
Dep. Variable:     Charging_Time_in_hours   R-squared:                       0.007
Model:                                GLS   Adj. R-squared:                  0.007
Method:                     Least Squares   F-statistic:                     1145.
Date:                    Fri, 31 Dec 2021   Prob (F-statistic):               0.00
Time:                            14:27:44   Log-Likelihood:            -6.4363e+06
No. Observations:                 1796287   AIC:                         1.287e+07
Df Residuals:                     1796275   BIC:                         1.287e+07
Df Model:                              11                                         
Covariance Type:                nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------

# OLS 

In [213]:
res_ols = sm.OLS(Y, X).fit()
print(res_ols.params)
print(res_wls.params)


const                   10.240982
pd_New                   0.074941
Precipitation_New       -0.043318
Temperature_New         -0.131111
Parking_New              0.060886
university_New          -0.004663
Biergarten_New          -0.036047
Kino_New                -0.036785
Restaurant_New          -0.122701
Community_Centre_New     0.076774
Hospital_New            -0.004524
Bahnhof_New              0.013312
dtype: float64
const                   10.240982
pd_New                   0.074941
Precipitation_New       -0.043318
Temperature_New         -0.131111
Parking_New              0.060886
university_New          -0.004663
Biergarten_New          -0.036047
Kino_New                -0.036785
Restaurant_New          -0.122701
Community_Centre_New     0.076774
Hospital_New            -0.004524
Bahnhof_New              0.013312
dtype: float64


In [214]:
se = np.vstack(
    [
        [res_wls.bse],
        [res_ols.bse],
        [res_ols.HC0_se],
        [res_ols.HC1_se],
        [res_ols.HC2_se],
        [res_ols.HC3_se],
    ]
)
se = np.round(se, 4)
colnames = ["x1", "const"]
rownames = ["WLS", "OLS", "OLS_HC0", "OLS_HC1", "OLS_HC3", "OLS_HC3"]
tabl = SimpleTable(se, colnames, rownames, txt_fmt=default_txt_fmt)
print(tabl)



          x1   const 
---------------------
WLS     0.0253 0.0009
OLS     0.0253 0.0009
OLS_HC0 0.0251 0.0009
OLS_HC1 0.0251 0.0009
OLS_HC3 0.0251 0.0009
OLS_HC3 0.0251 0.0009
---------------------


In [215]:
covb = res_ols.cov_params()
prediction_var = res_ols.mse_resid + (X * np.dot(covb, X.T).T).sum(1)
prediction_std = np.sqrt(prediction_var)
tppf = stats.t.ppf(0.975, res_ols.df_resid)



In [216]:
pred_ols = res_ols.get_prediction()
iv_l_ols = pred_ols.summary_frame()["obs_ci_lower"]
iv_u_ols = pred_ols.summary_frame()["obs_ci_upper"]

In [220]:
resid1 = res_ols.resid[w == 1.0]
var1 = resid1.var(ddof=int(res_ols.df_model) + 1)
resid2 = res_ols.resid[w != 1.0]
var2 = resid2.var(ddof=int(res_ols.df_model) + 1)
w_est = w.copy()
w_est[w != 1.0] = np.sqrt(var2) / np.sqrt(var1)
res_fwls = sm.WLS(Y, X, 1.0 / ((w_est ** 2))).fit()
print(res_fwls.summary())

IndexError: Boolean index has wrong length: 50 instead of 1796287

# WLS

In [154]:
data.exog = df[["pd_New","Precipitation_New","Temperature_New","Parking_New","university_New","Biergarten_New","Kino_New","Restaurant_New","Community_Centre_New","Hospital_New","Bahnhof_New",]]


In [155]:
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from scipy import stats
from statsmodels.iolib.table import SimpleTable, default_txt_fmt

In [ ]:
mod_wls = sm.WLS(y, X, weights=res_ols.params)
res_wls = mod_wls.fit()
print(res_wls.summary())
